# CSCI 3202, Spring 2018

# Friday 13 April 2018

# In-class notebook:  Markov Decision Processes (MDP)

<a id='top'></a>

<br>

* You will **not** submit this to Moodle. But it will be exceedingly useful for Homework 6 and the practicum to invest some time in understanding what is going on here.
* Even though you may be developing some key codes in groups here, it is expected on those future assignments that **all work is your own**.  

---

Before we begin, let's load a few packages that we might find useful.

In [1]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

---

<img src='https://roulette.casino/wp-content/uploads/2015/09/double-zero-roulette.png' style="width: 300px;"/>

## Kind Roulette

Suppose there is a betting game that has these rules.
* You start with some amount of money.
* You can wager any amount in increments of \$1, up to however much money you currently have. You cannot wager \$0.
* You flip a fair coin. If heads, you win your wager and your money increases by that amount. If tails, you lose your wager and your money decreases by that amount.
* You start playing this game with \$5, and that you will quit if you have at least \$10 at any point, or lose all your money. 

In typical American-style [roulette](https://en.wikipedia.org/wiki/Roulette), there are 18 red and 18 black bins, but also 2 green ones (0 and 00). Thus, the odds of winning a color-based bet are $18/38 \approx 0.473$. So this coin-flipping game is like a "kind" version of roulette.

<br>

### Part A: Play like a neanderthal

First, consider how you would play this game.  Think about it! Try playing a few rounds of this Kind Roulette game [right now](https://justflipacoin.com/).  I mean, not for actual money.  Do that ***after*** we solve for an optimal policy.

We will approach the problem of figuring out how to play this game by using **Markov decision processes**, which have as a strength the balance of ***risk versus reward***.

<br>

### Part B: Set-up

Create a list of the possible states, given these conditions.  The rules of the game dictate that you can never have negative money, so the lowest state is s=0.  What about the upper limit?  You will quit if you have more than \$10, but what if you had \$9, and then won your bet?

We can also create a list of all of the terminal states.  These include 0 and any states with value 10 or greater.

In [2]:
states = list(range(0, 19))
terminal_states = [0] + list(range(10, 19))

Define an `actions` function that takes an argument `s` for the current state, and returns a list of all the possible actions - or wager amounts - that we can make.  You may assume that `terminal_states` is implicitly known, so does not need to be fed in as an argument, and that `[None]` is the appropriate set of actions available to a terminal state.

**Example:** `action(2) = [1,2]`

In [3]:
def actions(s):
    '''Returns a list of the available actions from state s.
    That is, what wagers can we make?'''
    
    if s in terminal_states:
        return [None]
    
    possible_actions = []
    for i in range(0,s):
        possible_actions.append(i)
    return possible_actions

Define a `transition` function that takes arguments `s` for the current states and `a` for the current action and returns a list of tuples. The second element of each tuple is an "adjacent" state that can be reached from `s` by action `a`, and the first element is the probability of this transition occurring.

**Example:** `transition(2,1) = [(0.5, 1), (0.5, 3)]`

In [4]:
def transition(s, a):
    '''Returns the transition probabilities from state 
    s by action a, as a list of (probability, state) tuples.
    The default case below will handle a terminal state.'''
    if a is None:
        return [(0, s)]
    else:
        return [(0.5, s-a),(0.5, s+a)]
        

Finally, define a `reward` function that takes an argument of `s` for the current state, and returns the reward for that state. 

**First,** we need to choose rewards for the terminal states.
* If you win \$10 or more, then a natural choice is to return the amount of money that you have won. For example, `reward(11) = 11`. Return the profit.
* If you lose all \$5 that you started with, then you might think to return 0 as the reward. But really, you have actually *lost* \$5, so a better choice of reward is probably -5.

**Then,** we need to choose the default reward for being in a non-terminal state. The typical thing to do is incorporate a small negative reward, so that the agent is incentivized to find a winning state. But this neglects an important fact:  **people enjoy gambling**. So, a perfectly defensible choice of default reward might be small and positive. To start, let's use -0.01 as a reward for being in a non-terminal state.

In [5]:
def reward(s):
    '''Returns the reward for being in state s, either a
    terminal state reward, or the default non-terminal reward.'''
    
    if s in terminal_states:
        return s-5
    else:
        return -0.01

**Some notes:**  We can also incorporate into this reward structure more severe penalty for losing all of the money. That would incentivize an agent to try harder not to lose, and play more conservatively. Alternatively, gamblers might feel a greater joy at winning a larger payout than a smaller one, so we could make the rewards for larger payouts increase greater-than-linearly with the payout size.  We will play around with these later.

<br>

### Part C: Value iteration

Let's code up **value iteration** to solve for the utilities of each state under an optimal policy. Use a discount factor of 0.999 to reflect the fact that we can play many iterations of this game quickly, so future rewards are not really all that far away.

In [6]:
df = 0.999

Let's walk through a single iteration of value iteration before we tackle the real deal. First, we must initialize something to track the largest change in utility of states this iteration, and an initial `utility` for all states. We can use a dictionary for `utility`, to map states (keys) to their utility (values). Here, we initialize all states to have 0 utility initially. In the end, it won't matter what value we used for initialization.

In [7]:
max_change = 0
utility_old = {s : 0 for s in states}

We need to iterate over all states `s`, but let's start by just calculating the maximum expected utility for just one state.  Then, we can add a loop around our code. Only somewhat arbitrarily, we choose `s=2`:

In [8]:
s = 2

For each available action from state `s`, we need to know what states are possible to reach and what are their probabilities.  We need the output from `transition(s,a)` for each possible action from state `s`!

Here is one way we could get the set of next states and their probabilities, just for the action of betting \$1 (or \$2, depending on how you ordered the output):

In [9]:
next_states = transition(s, actions(s)[0])
print(next_states)

[(0.5, 2), (0.5, 2)]


If we want *all* of the next states and their probabilities under all of the actions possible from `s`, then we could use the following list comprehension:

In [10]:
next_states = [transition(s, a) for a in actions(s)]
print(next_states)

[[(0.5, 2), (0.5, 2)], [(0.5, 1), (0.5, 3)]]


`next_states` is a list-of-lists, where the first list corresponds to the original output that we had from our simpler call to `transition`, and the second list corresponds to the output of `transition(s, actions(s)[1])`.

Now we need to calculate the maximum expected utility among all the possible actions, as in Equation 17.6. So we can initialize a new utility dictionary to hold the updated utilities for each state.

In [11]:
utility_new = utility_old.copy()

Here, we can implement a nice little loop to find the best utility among the possible actions from state `s`. Complete the codes below to calculate the expected utility (`newsum`) for each set of probabilities and states in `next_state`. Then calculate `utility_new[s]` as in Equation 17.6 from the textbook (Bellman Equation), and update the `max_change`, if necessary.

In [12]:
best_utility = -999
for k in range(len(next_states)):
    newsum = 0
    for j in range(len(next_states[k])):
         
        exp_util += # utility_old[3] * next_states[k][j][0]
    best_utility = max(best_utility, exp_util)
utility_new[s] = # and here...
max_change = # ... and here...

SyntaxError: invalid syntax (<ipython-input-12-edcf967bfe8b>, line 6)

Below is gently modified pseudocode from the lecture slides. I removed the actual function definition so we spend more time here getting the actual algorithm correct, as opposed to wrestling with an `MDP` class. We actually have expressions for most of the things we need to fill in now, so have a crack at filling in the code, and iterating to find the optimal utilities.

When we actually iterate, we will need some convergence/exit criterion. That's where the tolerance `tol` and  Equation 17.8 from the textbook come into play:

$$\text{If}~~ |U_{i+1} - U_i| < \texttt{tol}\cdot \dfrac{1-\texttt{df}}{\texttt{df}},~~\text{then}~~ |U_{i+1} - U| < \texttt{tol}$$

What this tells us in plain English is that if some measure of difference between utilities on iteration $i$ and $i+1$ is less than $\text{tol}\cdot \dfrac{1-\text{df}}{\text{df}}$, then the error in the utility estimate in iteration $i+1$ is less than $\text{tol}$ itself.

Let's use an absolute error tolerance in utility of 0.001, and we can use `max_change` that we have been tracking as an estimate of $|U_{i+1} - U_i|$.  So we should exit our main iteration loop if $\texttt{max_change} <  \texttt{tol}\cdot \dfrac{1-\texttt{df}}{\texttt{df}}$

In [ ]:
# VALUE ITERATION:
df = 0.999
tol = 0.001

# initilize utility for all states

# iterate:

    # make a copy of current utility estimate, to be modified

    # initialize maximum change to 0

    # for each state s:

        # for each available action, what next states
        # are possible, and their probabilities?

        # calculate the maximum expected utility

        # new utility of s = reward(s) + 
        #                    discounted max expected utility

        # update maximum change in utilities, if needed

    # if maximum change in utility from one iteration to the
    # next is less than some tolerance, break!
        
# upon exit, utility_new is the utility of each state under an optimal policy

Finally, we need to compare the expected utility of all of the different actions available, so that we can choose the best one for each state:

$$\pi^*(s) = \underset{a \in \text{actions}(s)}{\arg\max} \sum_{s'} P(s' \mid s,a) U(s')$$

where $P$ we can get from the `transition` function, and $U$ is the utility we just found from value iteration. We can do this as follows:

In [ ]:
# initialize the policy for each state
policy = {s : None for s in states}

# loop over states to find the action that maximizes expected utility
for s in states:
    
    # initialize the best utility to something very bad, so we can improve it
    best_utility = (-999, None)
    
    # loop over actions, find which gives the highest expected utility
    for a in actions(s):
        
        # calculate the expected utility of action a from state s
        exp_util = sum([p*utility_new[s2] for p, s2 in transition(s,a)])
        
        # if this action has higher expected utility than the current best,
        # replace the best (utility, action) tuple with this one
        if exp_util > best_utility[0]:
            best_utility = (exp_util, a)
            
    # now we have the action (second element) that leads
    # to the highest expected utility (first element)
    policy[s] = best_utility[1]

# upon exit, policy has the optimal policy for each state
policy

<br>

### Part D:  That raised more questions than it answered.

Before proceeding, you may want to turn your value iteration and policy calculation routines into functions, and access your reward function down here for faster tinkering. But, you can also just leave things hard-coded and play around with the following:

**(1)**  In this default game, with reward (loss) of -5 for losing all your money, a reward of -0.01 for non-terminal states, and a discount factor of 0.999, what are the optimal moves in each state?

**(2)** Now suppose you enjoy recreational gambling, so you feel a positive reward of +0.01 for non-terminal states. If all else remains equal, what then is the optimal policy?

**(3)**  Reset the reward back to -0.01 for non-terminal states. In the "default" game, we assumed that the loss you feel by losing all \$5 that you started with is just the \$5. But in reality, when you lose all of your money gambling, you probably would feel even worse that if you spent it on food, or if someone had robbed you in a more straightforward manner.  Modify `reward(0)` so that your loss is \$10 instead.  How does this affect the optimal strategy for playing this game?

**(4)** Just as losing all \$5 might make you feel worse than -\$5, you might feel a larger-than-linear reward for winning some money. Modify the reward structure such that $\texttt{reward}(s) = s^2$ for $s \ge 10$. How does this affect the optimal strategy? 

**(5)** The probability of winning any bet is 18/38, or about 0.4737, for standard American-style roulette.  Try modifying the `transition(s, a)` function to represent roulette, and see how having the odds stacked against us might change our play strategy.

**(6)** A [Martingale strategy](https://en.wikipedia.org/wiki/Martingale_(betting_system)) is one in which the gambler would double their bet after each loss. In this way, when they eventually win their bet, they recover all previous losses, plus net a profit equal to the original bet. Modify this game (including the terminal states) to allow for this kind of strategy. Play around with how much money the gambler starts with and see how likely it is that they will make any profit (Note: this is not an MDP anymore because the policy is fixed, but it is an interesting problem.)